In [ ]:
from detectionmodel2 import Processor
import cv2
import os
import pandas as pd
from pathlib import Path

In [ ]:
def get_subdirectories(path):
    return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

In [ ]:
def get_filenames(directory):
    dir_path = Path(directory)
    return [f.name for f in dir_path.iterdir() if f.is_file()]

In [ ]:
def video_fall_detection(video_path, model):
    cap = cv2.VideoCapture(str(video_path))

    if not cap.isOpened():
        print("Cannot open video")
        exit()

    # Define the codec and create VideoWriter object
    #     fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #     out = cv2.VideoWriter('output.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    # Create an instance of Processor
    processor = Processor(model)

    while True:
        success, frame = cap.read()

        if not success:
            break

        # Process the frame using the Processor class
        frame = processor.process(frame)

        # Write the processed frame into the output video
    #         out.write(frame)

    # Display the processed frame
    #         cv2.imshow("Fall Detection", frame)
    #         if cv2.waitKey(1) & 0xFF == ord('q'):
    #             break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    # Return the statistics using the output_stats method from the Processor instance
    return processor.output_stats()

In [ ]:
def get_test_results(model,video_dir,fall_patterns_dir):
    complete_test_results = []
    for fall_pattern_path in fall_patterns_dir:
        print(fall_pattern_path)
        fall_pattern_full_path = Path(video_dir,fall_pattern_path)
        cam_angles_paths = get_filenames(fall_pattern_full_path)
        for cam_angles_path in cam_angles_paths:
            print(cam_angles_path)
            cam_angle_full_path = Path(fall_pattern_full_path,cam_angles_path)
            test_result = video_fall_detection(cam_angle_full_path,model)
            test_result['Angle'] = cam_angles_path[:-4]
            test_result['Fall Type'] = fall_pattern_path
            complete_test_results.append(test_result)
    df_results = pd.DataFrame(complete_test_results)
    df_results['model'] = model
    return df_results

In [ ]:
video_dir = Path(os.getcwd(),'./Videos')

In [ ]:
fall_patterns_dir = ['chute04']

In [ ]:
models = ['yolov8s-pose.pt']

In [ ]:
complete_df_results = pd.DataFrame()
backup_counter =  0
for model in models:
    print(model)
    df_result = get_test_results(model,video_dir,fall_patterns_dir)
    df_result.to_pickle(f'df_result_{backup_counter}.pkl')
    complete_df_results = pd.concat([complete_df_results,df_result]).reset_index(drop=True)
    backup_counter+=1
complete_df_results.to_pickle(f'complete_results_df.pkl')